# Generate national and state-level crosswalks
## 2000 block group parts to 2010 county

### NHGIS [block crosswalks](https://www.nhgis.org/user-resources/geographic-crosswalks)

**James D. Gaboardi, 06/2020**

In [1]:
%load_ext watermark
%watermark

2020-08-10T18:07:43-04:00

CPython 3.8.5
IPython 7.16.1

compiler   : Clang 10.0.1 
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
import nhgisxwalk
import inspect
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
numpy      1.19.1
nhgisxwalk 0.0.8
pandas     1.1.0



### Source and target years for the crosswalk

In [3]:
source_year, target_year = "2000", "2010"
gj_src, gj_trg = "GJOIN%s"%source_year, "GJOIN%s"%target_year

In [4]:
data_in = "../../crosswalks/"
data_tab = "../../tabular_data/"

### Source-target building base

In [5]:
base_xwalk_name = "nhgis_blk%s_blk%s_gj" % (source_year, target_year)
data_types = nhgisxwalk.str_types([gj_src, gj_trg])
from_csv_kws = {"path": data_in, "archived": True, "remove_unpacked": True}
read_csv_kws = {"dtype": data_types}
base_xwalk = nhgisxwalk.xwalk_df_from_csv(
    base_xwalk_name, **from_csv_kws, **read_csv_kws
)
base_xwalk.head()

,GJOIN2000,GJOIN2010,WEIGHT,PAREA
0,G01000100201001000,G01000100201002000,0.035897,0.008988
1,G01000100201001000,G01000100201002001,0.253330,0.263725
2,G01000100201001000,G01000100201002002,0.000000,0.000385
3,G01000100201001000,G01000100201002003,0.076297,0.055430
4,G01000100201001000,G01000100201002004,0.032441,0.007543


### Source summary data

In [6]:
base_source_name = "%s_block/%s_block.csv" % (source_year, source_year)
base_source_file = "%s%s" % (data_tab, base_source_name)
base_source_file

'../../tabular_data/2000_block/2000_block.csv'

### Convenience code shorthand/lookup

In [7]:
nhgisxwalk.valid_geo_shorthand(shorthand_name=False)

{'block': 'blk',
 'block group part': 'bgp',
 'block group': 'bg',
 'tract': 'tr',
 'county': 'co'}

### Set the `nhgisxwalk.GeoCrossWalk` parameters
##### see [nhgisxwalk.GeoCrossWalk](https://github.com/jGaboardi/nhgisxwalk/blob/92b4fe55de0a9c53d0315dcda8ec121faaf20aef/nhgisxwalk/geocrosswalk.py#L19) for full details

In [8]:
nhgisxwalk.desc_code_2000_SF1b

{'Persons': {'Persons': 'Universe',
  'NP001A': 'Source code',
  'FXS': 'NHGIS code',
  'Total': 'FXS001'},
 'Families': {'Families': 'Universe',
  'NP031A': 'Source code',
  'F2V': 'NHGIS code',
  'Total': 'F2V001'},
 'Households': {'Households': 'Universe',
  'NP010A': 'Source code',
  'FY4': 'NHGIS code',
  'Total': 'FY4001'},
 'Housing Units': {'Housing Units': 'Universe',
  'NH001A': 'Source code',
  'FV5': 'NHGIS code',
  'Total': 'FV5001'}}

In [9]:
input_vars = [
    nhgisxwalk.desc_code_2000_SF1b["Persons"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Families"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Households"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Housing Units"]["Total"]
]
input_vars

['FXS001', 'F2V001', 'FY4001', 'FV5001']

In [10]:
input_var_tags = ["pop", "fam", "hh", "hu"]

In [11]:
xwalk_args = {
    "source_year": source_year,
    "target_year": target_year,
    "source_geo": "bgp",
    "target_geo": "co",
    "base_source_table": base_source_file,
    "input_var": input_vars,
    "weight_var": input_var_tags,
    "keep_base": False,
    "add_geoid": True
}

### Generate data product
1. Create a national crosswalk then split by state 
2. Write out all products with `README.txt` files

In [12]:
nhgisxwalk.generate_data_product(base_xwalk, xwalk_args, data_in)

-----------------------------------------------